# Retail Segmentation Excercise

**RFM** is a method used for analyzing customer value. It is commonly used in database marketing and direct marketing and has received particular attention in retail and professional services industries.

RFM stands for the three dimensions:

* **R**ecency – _How recently did the customer purchase?_
* **F**requency – _How often do they purchase?_
* **M**onetary Value – _How much do they spend?_

In [175]:
!wget -q https://github.com/lexerdev/pairing-sessions/raw/main/data/orders.csv
!wget -q https://github.com/lexerdev/pairing-sessions/raw/main/data/customers.csv
!wget -q https://github.com/lexerdev/pairing-sessions/raw/main/data/order_items.csv

## Customer Data

This is a Brazilian ecommerce public dataset of orders made at [Olist Store](http://www.olist.com/). The dataset has information of 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil.

Its features allows viewing an order from multiple dimensions: from **order status**, price, payment and freight performance to customer location, product attributes and finally reviews written by customers. We also released a geolocation dataset that relates Brazilian zip codes to lat/lng coordinates.

This is real commercial data, it has been anonymised.

**Entity Relationship Diagram**

Below is an Entity Relationship Diagram of the available data:

![ERD](https://github.com/lexerdev/pairing-sessions/raw/main/images/erd.png)


**Data Dictionary**

**customers**

* `customer_id`:  Customer ID number
* `customer_unique_id`: Unique ID of the Customer from resolution
* `customer_zip_code_prefix`: Zip code of customer
* `customer_city`: City of customer
* `customer_state`: State of customer

**orders**


* `order_id`: Order ID number
* `customer_id`:  Customer ID number
* `order_status`: Status of order process
* `order_purchase_timestamp`: Time of the product is ordered
* `order_approved_at`: Time of payment for the order
* `order_delivered_carrier_date`: Time of notifying the logistics in order process
* `order_delivered_customer_date`: Time of the product arrived at the consumer
* `order_estimated_delivery_date`: Estimated arrival time of the order

**order_items**

* `order_item_id`: The number of the product in the order (ex：If there are three items in the order, the order_item_id is 1, 2, 3)
* `product_id`: Product ID number
* `seller_id `: Seller ID number
* `shipping_limit_date`: The deadline for the seller deliver the goods to the logistics
* `price`: Product price
* `freight_value`: delivery fee

In [176]:
import pandas as pd
import numpy as np

orders = pd.read_csv(
    "orders.csv",
    parse_dates=[
        "order_purchase_timestamp",
        "order_approved_at",
        "order_delivered_carrier_date",
        "order_delivered_customer_date",
        "order_estimated_delivery_date",
    ],
)
customers = pd.read_csv("customers.csv")
order_items = pd.read_csv("order_items.csv")

In [177]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [178]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [179]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


In [180]:
features = pd.DataFrame(customers['customer_unique_id'].unique(), columns = ['customer_unique_id'])
features.head()

,customer_unique_id
0,861eff4711a542e4b93843c6dd7febb0
1,290c77bc529b7ac935b93aa66c333dc3
2,060e732b5b29e8181a18229c7b0b2b5e
3,259dac757896d24d7702b9acbbff3f3c
4,345ecd01c38d18a9036ed96c73b8d066


In [181]:
from dateutil.parser import parse
recency = []
frequency = []
total_spend = []

In [182]:
for ind in features.index:
     temp = orders[orders.customer_id.isin(list(customers[customers.customer_unique_id == features['customer_unique_id'][ind]].customer_id))]
     most_recent_date = temp['order_approved_at'].max()
     d2 = parse('2018-12-31')
     if len(str(most_recent_date)) > 9:
         d1 = parse(str(most_recent_date))
         rec = round(abs((d2 - d1).days)/(365.25/12),2)
         d3 = d1 - datetime.timedelta(days=365)
         freq = len(temp[temp.order_approved_at >= d3])
         
     else:
         rec = 'NA'
         freq = 'NA'
     recency.append(rec)
     frequency.append(freq)
     temp2 = order_items[order_items.order_id.isin(list(temp.order_id))]
     total = round(temp2.price.sum(),2)
     total_spend.append(total)
     temp.iloc[0:0]

print(len(recency))
print(recency)
print()
print()
print(len(frequency))
print(frequency)
print()
print()
print(len(total_spend))
print(total_spend)

96096
[19.48, 11.56, 7.36, 9.59, 5.06, 14.49, 10.32, 13.44, 11.37, 11.7, 13.08, 10.71, 15.67, 6.8, 8.94, 11.01, 4.57, 6.41, 19.58, 13.93, 16.56, 14.49, 7.72, 20.53, 7.49, 10.64, 5.55, 19.52, 13.17, 7.16, 15.51, 13.08, 8.41, 18.46, 9.63, 7.72, 14.52, 16.66, 5.75, 10.58, 5.68, 12.39, 10.91, 14.69, 7.56, 8.38, 9.53, 8.41, 13.11, 23.1, 8.54, 7.26, 10.94, 7.66, 18.66, 10.84, 11.01, 17.64, 6.8, 9.07, 9.1, 10.28, 7.75, 20.86, 13.34, 14.95, 12.81, 15.57, 15.7, 5.03, 11.24, 9.33, 12.06, 15.11, 12.06, 21.26, 4.27, 4.17, 9.63, 11.56, 9.23, 19.29, 21.49, 6.77, 7.95, 14.09, 9.13, 6.41, 6.6, 9.3, 4.73, 6.83, 14.75, 8.51, 6.21, 8.02, 8.94, 16.1, 12.78, 4.24, 16.59, 12.09, 12.48, 21.09, 16.69, 9.13, 7.98, 6.01, 15.28, 17.48, 19.06, 22.74, 15.54, 12.25, 10.05, 6.87, 8.44, 16.62, 19.29, 22.64, 7.03, 16.95, 7.72, 6.6, 7.03, 6.93, 8.48, 7.13, 19.48, 22.51, 19.65, 18.2, 13.08, 15.93, 15.21, 12.68, 16.1, 12.98, 10.64, 15.08, 8.67, 8.57, 16.95, 4.63, 22.47, 19.19, 11.83, 4.24, 10.25, 7.75, 15.93, 4.99, 4.86,

In [183]:
print(features.size)

96096


In [184]:
features['recency'] = recency
features['frequency'] = frequency
features['total_spend'] = total_spend
features.head()

,customer_unique_id,recency,frequency,total_spend
0,861eff4711a542e4b93843c6dd7febb0,19.48,1,124.99
1,290c77bc529b7ac935b93aa66c333dc3,11.56,1,289.00
2,060e732b5b29e8181a18229c7b0b2b5e,7.36,1,139.94
3,259dac757896d24d7702b9acbbff3f3c,9.59,1,149.94
4,345ecd01c38d18a9036ed96c73b8d066,5.06,1,230.00


In [185]:
labels=[1,2,3,4,5,6,7,8,9,10]
features['monetary'] = pd.qcut(features['total_spend'], q = 10, labels=labels)
features.head()

,customer_unique_id,recency,frequency,total_spend,monetary
0,861eff4711a542e4b93843c6dd7febb0,19.48,1,124.99,7
1,290c77bc529b7ac935b93aa66c333dc3,11.56,1,289.00,10
2,060e732b5b29e8181a18229c7b0b2b5e,7.36,1,139.94,8
3,259dac757896d24d7702b9acbbff3f3c,9.59,1,149.94,8
4,345ecd01c38d18a9036ed96c73b8d066,5.06,1,230.00,9


In [186]:
features = features.drop(columns=['total_spend'])
features.head()

,customer_unique_id,recency,frequency,monetary
0,861eff4711a542e4b93843c6dd7febb0,19.48,1,7
1,290c77bc529b7ac935b93aa66c333dc3,11.56,1,10
2,060e732b5b29e8181a18229c7b0b2b5e,7.36,1,8
3,259dac757896d24d7702b9acbbff3f3c,9.59,1,8
4,345ecd01c38d18a9036ed96c73b8d066,5.06,1,9


In [187]:
features = features.replace(to_replace ="NA", value = 0)
features.head()

,customer_unique_id,recency,frequency,monetary
0,861eff4711a542e4b93843c6dd7febb0,19.48,1,7
1,290c77bc529b7ac935b93aa66c333dc3,11.56,1,10
2,060e732b5b29e8181a18229c7b0b2b5e,7.36,1,8
3,259dac757896d24d7702b9acbbff3f3c,9.59,1,8
4,345ecd01c38d18a9036ed96c73b8d066,5.06,1,9


## RFM Features


Let's see if we can setup a feature table like this:

| customer_unique_id | recency  | frequency | montetary |
|--------------------|----------|-----------|-----------|
| 9ef432eb...49a62d  | 2.591354 | 1.0       | 6         |
| ...                | ...      | ...       | ...       |

Where:
* `recency` is the months since the customer made a purchase from the last date of purchase in the orders dataset
* `frequency` is the distinct count of orders in the 365 days leading to the last date of purchase in the orders dataset
* `monetary` is a discretisation of `total_spend` in buckets between 1 and 10


Some guides are:
- Using [`np.timedelta64`](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-and-timedelta-arithmetic) is useful for determing the time between dates
- Using [`pd.qcut`](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html) is suitable for finding spend buckets
- Fill NA values with zero in the final feature table


What are the mean RFM values you found for all customers that have made a purchase?

In [188]:
features.astype({'recency': 'float'})
print(round(features['recency'].mean(),2))

11.87


In [189]:
features.astype({'frequency': 'int32'})
print(round(features['frequency'].mean(),2))

1.03


In [190]:
features = features.astype({'monetary': 'int32'})
print(round(features['monetary'].mean(),2))

5.49
